# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. Результаты исследования будут учтены при построении модели *кредитного скоринга* — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - ответить на четыре вопроса:
* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

На основании вышеизложенных вопросов выдвинем соответствующие **гипотезы**:
1. *Для возврата кредита в срок важен факт наличия хотя бы одного ребенка*
2. *Заемщики, находящиеся в официальном браке, более склонны к своевременному погашению кредита*
3. *Средний или высокий уровень дохода также напрямую увеличивает вероятность погашения кредита*
4. *Кредиты на образовательные цели и на покупку недвижимости с большей вероятностью будут выплачены в срок, нежели кредиты на автомобили и на проведение свадьбы*

**Ход исследования**

Данные о заемщиках мы получим из файла `/datasets/data.csv`. О качестве данных ничего не известно. Поэтому перед ответами на вопросы и проверкой гипотез понадобится обзор данных. 

Мы проверим данные на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки мы поищем возможность исправить самые критичные ошибки данных.
 
Таким образом, предварительно разделим исследование на три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

## Обзор данных

Составим первое впечатление о данных. Для этого сперва импортируем библиотеку `pandas` и библиотеку для обработки регулярных выражений `re` (она потребуется позднее):

In [1]:
import pandas as pd
import re

Прочитаем файл `data.csv` из папки `/datasets` и сохраним его в переменной `df`:

In [2]:
df = pd.read_csv('/datasets/data.csv')

Выведем на экран первые пять строк таблицы:

In [3]:
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Мы уже можем наблюдать аномалии - к примеру, отрицательное количество дней трудового стажа в столбце `days_employed` и сведения об образовании в графе `education`, написанные в разных регистрах.

Для обнаружения иных возможных артефактов в данных выведем также пять случайных строк датафрейма:

In [4]:
display(df.sample(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
652,0,NaN,32,среднее,1,Не женат / не замужем,4,F,сотрудник,0,NaN,на покупку подержанного автомобиля
6661,0,-440.129450,57,среднее,1,гражданский брак,1,F,сотрудник,0,198403.599805,свадьба
7616,0,-2371.014479,29,Среднее,1,женат / замужем,0,F,госслужащий,1,112699.942196,получение образования
6125,2,-2970.090619,33,среднее,1,женат / замужем,0,F,сотрудник,1,146768.230358,автомобиль
2381,1,-5281.453025,41,неоконченное высшее,2,женат / замужем,0,M,сотрудник,0,253341.200555,высшее образование


Получим общее представление о таблице:

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Как мы видим, в датафрейме одиннадцать столбцов, все поименованы в соответствии с общепринятым стилем. Представленные типы данных - `float`, `int` и `object`.

Согласно документации к данным:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Промежуточные выводы**

В каждой строке таблицы — данные о заёмщике. Всего представлено одиннадцать столбцов, что слишком объёмно и мешает восприятию информации. Хорошим решением станет разделить данные на три датафрейма: в одном будут содержаться данные для анализа и идентификаторы уровня образования и семейного положения, в двух других - "словари" с пояснениями к этим идентификаторам.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Однако в столбцах `days_employed` и `total_income` встречаются пропуски. Вероятно, они отражают людей без задокументированного опыта работы, - и это предположение также можно проверить уже на этапе предобработки.

Интерес представляют и типы данных - в уже упомянутых столбцах `days_employed` и `total_income` представлены вещественные значения. Для упрощения дальнейшего анализа их лучше будет перевести в целочисленный формат.

Как уже было указано выше, в данных присутствуют аномалии - к примеру, отрицательное количество дней трудового стажа в столбце `days_employed` и сведения об образовании в графе `education`, написанные в разных регистрах.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

### Заполнение пропусков
Посчитаем, сколько в таблице пропущенных значений:

In [6]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Мы встретились с пропусками только в столбцах `days_employed ` и `total_income`, причем их количество совпадает. Проверим, есть ли между ними взаимосвязь - т.е. пересекаются ли строчки с отсутствующими значениями между собой:

In [7]:
df.loc[(df.days_employed.isna()) & (df.total_income.isna())]['income_type'].count()

2174

Как видно выше, взаимосвязь прямая - у заемщиков с отсутствующим в графе стажем работы размер дохода также не представлен. Чтобы определить, случайными ли являются пропуски или нет, проверим тип занятости этих заемщиков. 
Для этого выведем последние десять значений:

In [8]:
df.loc[(df.days_employed.isna()) & (df.total_income.isna())]['income_type'].tail(10)

21415    пенсионер
21423    пенсионер
21426    сотрудник
21432    сотрудник
21463    сотрудник
21489    компаньон
21495    сотрудник
21497    компаньон
21502    сотрудник
21510    сотрудник
Name: income_type, dtype: object

Поскольку представлено несколько типов занятости, то отсутствие значений, как мы и предполагали, носит случайный характер. В этом случае данные отсутствуют по технологическим причинам. Увы, восстановить значения в рамках учебного проекта не представляется возможным.

Подойдем к проблеме с другой стороны. Пропуски составляют одну десятую часть от всего набора данных, что довольно много. Если сбросить все строки с пропусками, то мы рискуем исказить правдоподобность данных, что в итоге скажется на работе модели кредитного скоринга. Значит, остается только вариант заполнения недостающих данных.

Проверим степень разброса значений в столбцах `days_employed` и `total_income`. Для упрощения визуализации сгруппируем данные относительно пола и применим агрегатную функцию `agg()`:

In [9]:
df.groupby('gender').agg({
    'days_employed': ['min', 'max'],
    'total_income': ['min', 'max']
})

days_employed                  total_income              
                 min            max            min           max
gender                                                          
F      -18388.949901  401755.400475   20667.263793  1.715018e+06
M      -15267.541183  401674.466633   21205.280566  2.265604e+06
XNA     -2358.600502   -2358.600502  203905.157261  2.039052e+05

Сразу же обратим внимание на аномалии в `days_employed`, которые будем прорабатывать далее:
* уже отмеченные в **пункте 1** отрицательные значения стажа;
* максимальные значения стажа, не соответствующие действительности (401755 дней равносильны 1100 годам).

Итак, в столбцах `days_employed` и `total_income` содержатся количественные значения с сильным разбросом от минимума до максимума. Самым лучшим вариантом будет заполнение пропусков медианным значением. 

Эти значения мы примем в разрезе значений столбца `income_type` - для каждого типа занятости мы рассчитаем медианный доход `total_income` и медианное количество дней стажа `days_employed`, и далее заполним пропуски ими. Так мы не усредним данные и меньше повлияем на распределение.

Для обоснования выведем на экран данные по медианам для каждого типа занятости, используя агрегатную функцию `agg()`:

In [10]:
df.groupby('income_type').agg({
    'days_employed': 'median',
    'total_income': 'median'
})

,days_employed,total_income
income_type,,
безработный,366413.652744,131339.751676
в декрете,-3296.759962,53829.130729
госслужащий,-2689.368353,150447.935283
компаньон,-1547.382223,172357.950966
пенсионер,365213.306266,118514.486412
предприниматель,-520.848083,499163.144947
сотрудник,-1574.202821,142594.396847
студент,-578.751554,98201.625314


Мы видим, что наш медианные значения для разных типов занятости сильно разнятся - т.е. подход обоснован.

Заполним пропуски в `days_employed` и `total_income` медианным значением по каждому типу занятости. Используем для этого метод `fillna()` совместно с группировкой и методом `transform()`. Последний позволит нам модифицировать медианные значения под каждый из типов занятости:

In [11]:
df['days_employed'] = df.days_employed.fillna(df.groupby(['income_type'])['days_employed'].transform('median'))
df['total_income'] = df.total_income.fillna(df.groupby(['income_type'])['total_income'].transform('median'))

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения:

In [12]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Проверка данных на аномалии и исправления
Мы уже неоднократно отмечали аномалии в столбце `days_employed`. В **пункте 2.1** упоминалось, что артефакты в этом столбце могут быть вызваны технологическими причинами: некорректной выгрузкой результатов в файл или ошибками при чтении файла.

Сперва обработаем отрицательные значения. Данные столбца относятся к типу `float`, поэтому здесь мы можем применить метод `abs()`, возвращающий абсолютное значение числа:

In [13]:
df['days_employed'] = df.days_employed.abs()

Для самопроверки выведем пять первых строк таблицы:

In [14]:
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Посмотрим, изменились ли медианные данные столбца для каждого типа занятости. Используя группировку и агрегатную функцию `agg()`, выведем их на экран:

In [15]:
df.groupby('income_type').agg({
    'days_employed': 'median'
})

,days_employed
income_type,
безработный,366413.652744
в декрете,3296.759962
госслужащий,2689.368353
компаньон,1547.382223
пенсионер,365213.306266
предприниматель,520.848083
сотрудник,1574.202821
студент,578.751554


При визуальном сравнении с похожей группировкой в **пункте 2.1** видно, что медианы изменились только относительно знака - из отрицательных они стали положительными. Сами же значения остались неизменными. Это говорит о том, проводить изменения по присвоению новых медиан не требуется.

Теперь обратим внимание непосредственно на сами значения стажа. Мы уже заметили, что максимум в этом столбце эквивалентен 1100 годам, что противоречит действительности.

Предположим, максимальный стаж работы для мужчин и женщин - 45 лет (с 18 лет до 63 лет). Это равно 16425 дням. Выведем на экран количество значений, выходящих за эти рамки, при помощи метода `count()`:

In [16]:
df.loc[df.days_employed > 16425, 'days_employed'].count()

3861

Подобные аномалии составляют пятую часть от всего датасета - это очень много. Необходимо ввести соответствующие корректировки. Поскольку стаж работы в наших гипотезах не фигурирует и его значения не будут участвовать в анализе, ограничимся заменой этих аномалий на медианное значение по столбцу:

In [17]:
df.loc[df.days_employed > 16425, 'days_employed'] = df['days_employed'].median()

Вновь выведем сгруппированные по типам занятости показатели стажа работы:

In [18]:
df.groupby('income_type').agg({
    'days_employed': 'median'
})

,days_employed
income_type,
безработный,1993.522017
в декрете,3296.759962
госслужащий,2689.368353
компаньон,1547.382223
пенсионер,1993.522017
предприниматель,520.848083
сотрудник,1574.202821
студент,578.751554


На данном этапе результат нас устраивает. Но следует учесть, что если возникнет необходимость анализа показателей в зависимости от стажа работы, то аномальные максимумы в столбце `days_employed` разумнее будет заменить на медианные значения в разрезе, к примеру, соотносимого показателя `dob_years`.

Далее, с помощью метода `value_counts()` проверим на наличие аномалий столбец `children`:

In [19]:
df.children.value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Тут наблюдается информация, явно противоречащая действительности:
* отрицательное количество детей
* двадцать детей у отдельно взятого родителя

Последнее могло бы иметь место быть, но в единичном случае. В наших же данных представлено семьдесят шесть семей с двадцатью детьми, что приводит к мысли, что данные были внесены ошибочно.

Заменим отрицательные значения в столбце `children` на положительные:

In [20]:
df['children'] = df.children.abs()

Предположим, что значение '20' в столбце `children` - это некорректно внесённое значение '2', и нивелируем аномалию исходя из этого:

In [21]:
df.loc[df.children == 20, 'children'] = 2

Для проверки внесенных изменений вновь выведем на экран количество значений:

In [22]:
df.children.value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Теперь изучим на предмет аномалий значения в столбце `gender`:

In [23]:
df.gender.value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Здесь мы наблюдаем один случай артефакта, где пол заёмщика не определён. Поскольку в данных не представлены ФИО, определить, к чему его отнести, не представляется возможным - однако это и не важно, т.к. эта аномалия встречается единожды и на статистику не повлияет. 

Однако, стоит отметить иную особенность - количество заёмщиков-женщин вдвое превышает количество заёмщиков-мужчин. Это - серьёзное препятствие проведению анализа по этому показателю. Если бы мы захотели провести подобное исследование, то необходимо было бы обогатить данные так, чтобы распределение по категориям "мужчина-женщина" стало равномерным.

Далее рассмотрим столбец `dob_years`. Поскольку переменная является количественной, логичнее всего будет вывести её минимальное и максимальное значение при помощи функции `agg()`, предварительно сгруппировав данные по типам занятости:

In [24]:
df.groupby('income_type').agg({
    'dob_years': ['min', 'max']
})

dob_years    
                      min max
income_type                  
безработный            31  45
в декрете              39  39
госслужащий             0  75
компаньон               0  74
пенсионер               0  74
предприниматель        27  58
сотрудник               0  74
студент                22  22

Нам встретились нулевые значения возраста заемщика - это явная аномалия. Выясним их количество, подсчитав их методом `count()`:

In [25]:
df[df.dob_years == 0]['dob_years'].count()

101

Их количество сравнительно мало - примерно 0.5% из всей выборки данных - и не является статистически значимым показателем.

С другой стороны, если бы в нашем исследовании участвовал количественный показатель возраста заёмщика, то было бы предпочтительным заменить эту аномалию на медианное значение по группе. На данном же этапе мы предпочтём не вносить в этот параметр никаких изменений.

Больше аномалий не наблюдается, и мы можем двигаться дальше.

### Изменение типов данных.
Теперь рассмотрим столбцы `days_employed` и `total_income`. В них отражены данные вещественного типа, но формат их представления не слишком удобен:
* трудовой стаж в столбце `days_employed` указан в днях, и его лучше представить как целочисленные значения
* ежемесячный доход в `total_income` как финансовые данные предпочтительнее было бы указать с точностью до сотых, но в нашем случае для упрощения дальнейшего анализа мы также приведем к целочисленным значениям

Исправим это при помощи метода `astype()`:

In [26]:
df['days_employed'] = df.days_employed.astype('int')
df['days_employed'] = df.total_income.astype('int')

Проверим изменения на выводе типов данных в этих столбцах датафрейма:

In [27]:
df[['days_employed', 'days_employed']].dtypes

days_employed    int64
days_employed    int64
dtype: object

### Удаление дубликатов.
При знакомстве с данными нам уже встречались неявные дубликаты в столбце `education`.
Выведем количество уникальных значений в этом столбце и связанном с ним столбце `education_id` при помощи метода `nunique()`:

In [28]:
print(f'Количество уникальных значений education_id: {df.education_id.nunique()}')
print(f'Количество уникальных значений education: {df.education.nunique()}')

Количество уникальных значений education_id: 5
Количество уникальных значений education: 15


Налицо явная несостыковка: количество идентификаторов и уровня образования должно совпадать.
Уточним, какие уникальные значения содержатся в столбце `education`:

In [29]:
df.education.sort_values().unique()

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

По всей видимости, данные об образовании вносились вручную - по этой причине мы видим разницу в регистрах. Чтобы привести данные к общему виду, воспользуемся методом `.lower()` для строк:

In [30]:
df['education'] = df.education.str.lower()

Проверим количество оставшихся уникальных значений в этом столбце:

In [31]:
df.education.nunique()

5

Теперь количество идентификаторов и уровня образования совпадает.

Проведем аналогичную проверку для столбцов `family_status_id` и `family_status`:

In [32]:
print(f'Количество уникальных значений family_status_id: {df.family_status_id.nunique()}')
print(f'Количество уникальных значений family_status: {df.family_status.nunique()}')

Количество уникальных значений family_status_id: 5
Количество уникальных значений family_status: 5


Относительно этих двух столбцов нареканий нет.

Теперь, когда данные приведены к общему виду, проверим их на содержание явных дубликатов при помощи метода `duplicated()`:

In [33]:
df.duplicated().sum()

71

Сбросим дублированные значения с помощью метода `drop_duplicates()` c присваиванием новых индексов:

In [34]:
df = df.drop_duplicates().reset_index(drop=True)

Ещё раз посчитаем явные дубликаты в таблице и убедимся, что полностью от них избавились:

In [35]:
df.duplicated().sum()

0

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.
В промежуточных выводах к **пункту 1** указывалось, что для упрощения восприятия данных предпочтительнее будет разделить изначальный датафрейм на три таблицы.

Создадим два новых датафрейма, в которых:
* каждому уникальному значению из `education` соответствует уникальное значение `education_id` — в первом;
* каждому уникальному значению из `family_status` соответствует уникальное значение `family_status_id` — во втором.

Создадим из исходного датафрейма "словарь" `education_df` и выведем на экран первые пять его строк:

In [36]:
education_df = df[['education_id', 'education']]
display(education_df.head())

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее


Нам встретились дубликаты. Сбросим их и выведем "словарь" `education_df` на экран:

In [37]:
education_df = education_df.drop_duplicates().reset_index(drop=True)
display(education_df)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


К этому "словарю" замечаний нет.

Приступим к формированию "словаря" `family_status_df`:

In [38]:
family_status_df = df[['family_status_id', 'family_status']]
display(family_status_df.head())

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак


Удалим продублированные значения, оставив только уникальные, и выведем "словарь" `family_status_df` на экран:

In [39]:
family_status_df = family_status_df.drop_duplicates().reset_index(drop=True)
display(family_status_df)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Теперь произведём декомпозицию исходного датафрейма `df`. Удалим столбцы `education` и `family_status`, оставив только их идентификаторы: `education_id` и `family_status_id`. Новые датафреймы станут «словарями», к которым мы сможем обращаться по идентификатору:

In [40]:
df = df[['children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose']]

Выведем пять первых строк обновлённого датафрейма на экран:

In [41]:
display(df.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,253875,42,0,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,112080,36,1,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,145885,33,1,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,267628,32,1,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,158616,53,1,1,F,пенсионер,0,158616.077870,сыграть свадьбу


На этом первая часть предобработки подошла к концу. В следующих двух пунктах мы займёмся разнесением данных по категориям.

### Категоризация дохода
В нашем датафрейме, в столбцах `total_income` и `purpose` присутствуют данные, имеющие определенное значение признака, которое встречается в наборе данных однократно. Это вполне логично: значение ежемесячного дохода (количественная переменная), как и цель получения кредита (категориальная переменная) индивидуальны для каждого заёмщика. Но делать статистические выводы по таким значениям нельзя. Поэтому с такими данными проводится *категоризация* - т.е. объединение данных по категориям.

Начнём с категоризации дохода. 
Для ориентира возьмём среднерыночные показатели заработной платы и исходя из них определим категории. Пропишем функцию с ежемесячным доходом в качестве параметра, которая будет присваивать доходам категории на основании диапазонов, указанных ниже:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

Например, кредитополучателю с доходом 25000 нужно назначить категорию 'E', а клиенту, получающему 235000, — 'B'.

In [42]:
# определим функцию set_income_category, присваивающую категорию доходу
def set_income_category(income):
    # используем конструкцию try...except для обработки возможных ошибок
    try:
        # условие присвоения категории E
        if 0 <= income <= 30000:
            return 'E'
        # условие присвоения категории D
        elif income <= 50000:
            return 'D'
        # условие присвоения категории C
        elif income <= 200000:
            return 'C'
        # условие присвоения категории B
        elif income <= 1000000:
            return 'B'
        # условие присвоения категории A
        elif income > 1000001:
            return 'A'
        # условие, если доход меньше нуля
        elif income < 0:
            return 'income is negative'
    # блок except, исключающий случай, когда значение в income относится к строковому типу
    except TypeError:
        return 'undefined'
# если в результатах работы функции будут присутствовать категории "income is negative" и "undefined",
# mo это повод провести повторно пункт 2.2 и пункт 2.3 предобработки данных, соответственно

На основании функции `set_income_category` создадим в таблице столбец `total_income_category`:

In [43]:
df['total_income_category'] = df.total_income.apply(set_income_category)

Выведем пять случайных строк обновлённого датафрейма на экран:

In [44]:
display(df.sample(5))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
6887,0,56516,57,1,2,F,пенсионер,0,56516.608407,покупка коммерческой недвижимости,C
1327,0,131592,49,1,1,F,сотрудник,0,131592.226862,сыграть свадьбу,C
20829,0,253236,47,0,4,F,компаньон,0,253236.120663,строительство недвижимости,B
15348,0,298150,38,0,0,M,компаньон,0,298150.028437,строительство собственной недвижимости,B
2334,0,78500,56,1,0,F,пенсионер,0,78500.291635,сделка с подержанным автомобилем,C


Проверим, встречаются ли категории дохода `income is negative` или `undefined`. В случае их присутствия рекомендуется не переходить к следующему шагу и вернуться к **пункту 2.2** и **пункту 2.3** предобработки данных, дабы не допустить падения кода с ошибкой. 
Сделать проверку поможет метод `value_counts()`:

In [45]:
df.total_income_category.value_counts()

C    16015
B     5042
D      350
A       25
E       22
Name: total_income_category, dtype: int64

Вышеназванные категории нам не встретились, и мы можем двигаться дальше.

Для визуализации результатов создадим сводную таблицу при помощи метода `pivot_table()`. 
Передадим ей следующие аргументы:
* `income_type`- в качестве индексов
* `total_income_category`- в качестве наименований столбцов
* `total_income` - в качестве значений, которые будут отражены в таблице
* `mean()` - в качестве агрегатной функции, которая будет применена к значениям таблицы

В последнем пункте мы предпочли использовать метод расчёта среднего арифметического `mean()` вместо метода подсчеты медианы `median()`, поскольку значения дохода в рамках категорий находятся в ожидаемых пределах и колебания в них минимальны:

In [46]:
income_pivot = df.pivot_table(index='income_type',
                             columns='total_income_category',
                             values='total_income',
                             aggfunc='mean')
display(income_pivot)

total_income_category,A,B,C,D,E
income_type,,,,,
безработный,NaN,202722.511368,59956.991984,NaN,NaN
в декрете,NaN,NaN,53829.130729,NaN,NaN
госслужащий,NaN,289638.274422,129891.912590,43862.783870,29200.077193
компаньон,1.380527e+06,301821.050392,141230.931572,44473.893177,28702.812889
пенсионер,NaN,277416.431715,115392.218670,42368.730703,25469.602108
предприниматель,NaN,499163.144947,NaN,NaN,NaN
сотрудник,1.277005e+06,276890.546244,127596.007685,43088.656187,25510.331556
студент,NaN,NaN,98201.625314,NaN,NaN


Исходя из сводной таблицы, можно понять, какими уровнями дохода могут обладать заёмщики с тем или иным типом занятости.
Кому-то присвоена только одна категория - как в случае студентов (категория 'C') или лиц в декрете (также категория 'C'). А кто-то, напротив, имеет разброс категорий дохода от 'A' до 'E' включительно - к примеру, сотрудники и компаньоны.

Эти данные могут пригодиться нам в дальнейшем, когда мы будем проверять гипотезы.

### Категоризация целей кредита

Прежде, чем присваивать категории целям кредита, изучим данные в столбце `purpose` и определим, на какие подстроки будет лучше ориентироваться. В этом нам поможет уже знакомый метод `value_counts()`:

In [47]:
df.purpose.value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Информация в этом столбце вносилась, по всей видимости, со слов заёмщика - по этой причине присутствует так много похожих вариантов. Кроме того, данные относятся к категориальному типу. Поэтому наилучшим решением будет определение подходящих категорий. 
Исходя из представленных данных, можно выделить четыре основных типа:
* 'операции с автомобилем'
* 'проведение свадьбы'
* 'получение образования'
* 'операции с недвижимостью'

С целью упрощения процесса используем поиск по шаблону при помощи метода `search()` ранее импортированного модуля `re`. Метод `search()` ищет в строке (второй параметр функции) совпадения со словом (первый параметр функции). Но вместо конкретного слова нужно указать шаблон с т.н. подстановочным знаком (или wildcard). Так, к примеру, шаблон *'автомобил?'* отражает не только само слово 'автомобиль', но и все остальные случаи его употребления - 'автомобиля', 'автомобилем' и т.д. В нашем случае подстановочным знаком будет выступать `?`, т.к. нам требуется любая одна или более букв, стоящая после заданного в шаблоне слова.

Определим функцию `set_purpose_category()`, которая будет присваивать соответствующую категорию в зависимости от цели в столбце `purpose`:

In [48]:
# определим функцию set_purpose_category, присваивающую категорию цели кредита
def set_purpose_category(purpose):
    # используем конструкцию try...except для обработки возможных ошибок
    try:
        # поиск слова 'автомобиль' по шаблону при помощи search()
        # условие присвоения категории 'операции с автомобилем'
        if re.search('автомобил?', purpose):
            return 'операции с автомобилем'
        # поиск слова 'свадьба' по шаблону при помощи search()
        # условие присвоения категории 'проведение свадьбы'
        elif re.search('свадьб?', purpose):
            return 'проведение свадьбы'
        # поиск слова 'образование' по шаблону при помощи search()
        # условие присвоения категории 'получение образования'
        elif re.search('образовани?', purpose):
            return 'получение образования'
        # поиск слова 'жилье' или слова 'недвижимость' по шаблону при помощи search()
        # условие присвоения категории 'операции с недвижимостью'
        elif (re.search('жиль?', purpose)) or (re.search('недвижимост?', purpose)):
            return 'операции с недвижимостью'
        # условие при значении, не попадающем в критерии выше
        else:
            return 'uncategorized'
    # блок except, исключающий случай, когда значение в income относится к числовому типу
    except TypeError:
        return 'undefined'

На основании функции `set_purpose_category` создадим в таблице столбец `purpose_category`:

In [49]:
df['purpose_category'] = df.purpose.apply(set_purpose_category)

Выведем пять случайных строк обновлённого датафрейма на экран:

In [50]:
display(df.sample(5))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
14315,0,620529,49,0,0,M,госслужащий,0,620529.352931,операции с коммерческой недвижимостью,B,операции с недвижимостью
6274,1,96988,35,1,0,F,сотрудник,0,96988.956279,приобретение автомобиля,C,операции с автомобилем
10477,2,187448,46,0,0,M,компаньон,0,187448.319964,операции с коммерческой недвижимостью,C,операции с недвижимостью
821,0,313621,39,0,0,F,компаньон,0,313621.187264,строительство жилой недвижимости,B,операции с недвижимостью
8388,0,202312,51,1,0,M,сотрудник,0,202312.629076,покупка недвижимости,B,операции с недвижимостью


Проверим при помощи `value_counts()`, встречаются ли категории `uncategorized` или `undefined`:

In [51]:
df.purpose_category.value_counts()

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

Вышеназванных категорий нам не встретилось - значит, наше разделение на четыре типа было обоснованным, и добавлять категории не потребуется. К тому же, с подобной категоризацией пропадает необходимость содержания столбца `purpose` в основном датафрейме, и мы можем провести декомпозицию, аналогичную **пункту 2.5**:

In [52]:
purpose_df = df[['purpose', 'purpose_category']]

df = df[['children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'total_income_category', 'purpose_category']]

Выведем на экран первые строки 'словаря' `purpose_df`:

In [53]:
display(purpose_df.head())

,purpose,purpose_category
0,покупка жилья,операции с недвижимостью
1,приобретение автомобиля,операции с автомобилем
2,покупка жилья,операции с недвижимостью
3,дополнительное образование,получение образования
4,сыграть свадьбу,проведение свадьбы


И выведем первые строки обновлённого датафрейма `df`:

In [54]:
display(df.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,total_income_category,purpose_category
0,1,253875,42,0,0,F,сотрудник,0,253875.639453,B,операции с недвижимостью
1,1,112080,36,1,0,F,сотрудник,0,112080.014102,C,операции с автомобилем
2,0,145885,33,1,0,M,сотрудник,0,145885.952297,C,операции с недвижимостью
3,3,267628,32,1,0,M,сотрудник,0,267628.550329,B,получение образования
4,0,158616,53,1,1,F,пенсионер,0,158616.077870,C,проведение свадьбы


Проводить аналогичную декомпозицию со столбцами `total_income` и `total_income_category` в **пункте 2.6**  не имеет особого смысла, т.к. в этом случае мы потеряем прямой доступ к расчётным данным в лице `total_income` и процесс анализа будет затруднён.

Теперь визуализируем результаты. Создадим сводную таблицу при помощи метода `pivot_table()`. 
Передадим ей следующие аргументы:
* `income_type`- в качестве индексов
* `purpose_category`- в качестве наименований столбцов
* `total_income` - в качестве значений, которые будут отражены в таблице
* `mean()` - в качестве агрегатной функции, которая будет применена к значениям таблицы

Метод `mean()` был выбран по причине, аналогичной причине, указанной в **пункте 2.6**:

In [55]:
purpose_pivot = df.pivot_table(index='income_type',
                             columns='purpose_category',
                             values='total_income',
                             aggfunc='mean')
display(purpose_pivot)

purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
income_type,,,,
безработный,NaN,131339.751676,NaN,NaN
в декрете,53829.130729,NaN,NaN,NaN
госслужащий,167519.201968,168317.340644,170505.104124,171204.086554
компаньон,201275.930625,202543.002371,188835.106505,200091.536920
пенсионер,133553.726002,134913.528145,136138.185291,138445.807202
предприниматель,NaN,499163.144947,NaN,499163.144947
сотрудник,159100.948861,160894.961905,158802.712226,155575.858421
студент,NaN,98201.625314,NaN,NaN


Здесь мы можем наблюдать интересную закономерность. В то время, как у госслужащих, компаньонов, пенсионеров и сотрудников цели кредитования разнятся, иные категории граждан берут кредиты только на конкретные цели. Например:
* граждане без работы и студенты сосредоточены исключительно на покупке жилья,
* предприниматели - на приобретении недвижимости и проведении свадьбы,
* люди в декретном отпуске - на покупке автомобиля.

**Промежуточные выводы**

Предобработка выявила следующие проблемы в данных:
* пропущенные значения
* аномалии (отрицательные значения)
* неудобный тип представления данных
* дубликаты - явные и неявные

Мы откорректировали аномалии, превратив их в абсолютные значения, и изменили типы данных, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным. Пропуски мы заменили на медианные значения с целью сохранения правдоподобности данных.

Кроме того, для упрощения дальнейшего анализа было принято решение о декомпозиции исходного датафрейма и выделения из него вспомогательных "словарей" с расшифровками.

Также мы разнесли данные по доходам и целям по соответствующим категориям и визуализировали результаты в виде сводных таблиц. В случае целей кредитования мы провели дополнительную декомпозицию датафрейма с выделением соответствующего "словаря".

Уже на этапе предобработки мы наблюдали некоторые закономерности в данных:
* относительно доходов:
1. Доходы студентов и лиц в декрете относятся к категории 'C' (от пятидесяти до двухсот тысяч включительно).
2. Доходы лиц без работы относятся к двум категориям - 'C' и 'B' (т.е. от пятидесяти тысяч до одного миллиона включительно). Это может говорить как о неправдоподобности предоставления данных заёмщиками, так и о наличии дополнительных, не указанных в таблице источников дохода. 
    
    В любом случае, по представленным данным сложно судить однозначно, необходим запрос дополнительной информации.

* относительно целей:
1. Граждане без работы и студенты сосредоточены исключительно на покупке жилья.
2. Люди в декретном отпуске заинтересованы в покупке автомобиля.
3. Предприниматели берут кредиты на приобретение недвижимости и проведение свадьбы.

Прежде, чем приступать к проверке наших гипотез, необходимо осуществить ещё один дополнительный шаг - и выяснить, можно ли опираться на данные столбца `income_type` в своих выводах.

## Исследование типов занятости

Для того, чтобы понять, являются ли данные в столбце `income_type` статистически значимыми, посмотрим на распределение его значений. Сделаем это при помощи метода `value_counts()`:

In [56]:
df.income_type.value_counts()

сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

**Промежуточные выводы**

Мы видим, что некоторые категории заёмщиков - предприниматели, безработные, лица в декрете и студенты - представлены в крайне малом количестве относительно всех остальных групп граждан. Это означает, что при имеющихся данных включать в дальнейший анализ все типы занятости не имеет смысла - в результате мы получим искаженные данные, противоречащие статистике.

Если мы хотим в дальнейшем проводить анализ с участием типов занятости, то лучшим решением будет уточнение и обогащение набора данных так, чтобы эти четыре категории заёмщиков были представлены в более полном виде.

## Проверка гипотез

### Влияние количества детей на возврат кредита

Напомним, первая гипотеза звучит следующим образом:

*Для возврата кредита в срок важен факт наличия хотя бы одного ребенка*.

Проверим первое предположение при помощи сводной таблицы. Передадим методу `pivot_table()` следующие аргументы:
* `children`- в качестве индексов
* `debt`- в качестве наименований столбцов
* `dob_years` - в качестве значений, которые будут отражены в таблице
* `count()` - в качестве агрегатной функции, которая будет применена к значениям таблицы

Таким образом мы рассчитаем количество людей с разным количеством детей, имеющих или не имеющих банковские задолженности:

In [57]:
children_pivot = df.pivot_table(index='children',
                            columns='debt',
                            values='dob_years',
                            aggfunc='count')
display(children_pivot)

debt,0,1
children,,
0,13028.0,1063.0
1,4410.0,445.0
2,1926.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


#### Выводы:

Данные демонстрируют следующие особенности:
* у подавляющего большинства заёмщиков нет детей и нет задолженностей;
* примерно одна десятая часть всех заёмщиков имеет задолженности, вне зависимости от количества детей;
* у заёмщиков с пятью детьми задолженности отсутствуют - но общее количество таких людей крайне мало и не может служить ориентиром.

Таким образом, данную гипотезу можно полностью опровергнуть.

### Влияние семейного положения на возврат кредита

Проверим вторую гипотезу:

*Заемщики, находящиеся в официальном браке, более склонны к своевременному погашению кредита*.

Осуществим это также при помощи сводной таблицы. Передадим методу `pivot_table()` следующие аргументы:
* `family_status_id`- в качестве индексов
* `debt`- в качестве наименований столбцов
* `dob_years` - в качестве значений, которые будут отражены в таблице
* `count()` - в качестве агрегатной функции, которая будет применена к значениям таблицы

Таким образом мы рассчитаем количество людей с разным семейным положением, имеющих или не имеющих банковские задолженности:

In [58]:
family_pivot = df.pivot_table(index='family_status_id',
                            columns='debt',
                            values='dob_years',
                            aggfunc='count')
display(family_pivot)

debt,0,1
family_status_id,,
0,11408,931
1,3763,388
2,896,63
3,1110,85
4,2536,274


Мы видим, что подавляющее большинство людей, не имеющих задолженностей по кредиту, относятся с категории '0' в графе `family_status_id`. Запросим информацию по этому идентификатору из "словаря" `family_status_df`:

In [59]:
family_status_df.loc[family_status_df.family_status_id == 0, 'family_status']

0    женат / замужем
Name: family_status, dtype: object

Также запросим информацию по категории '1' из "словаря" `family_status_df`:

In [60]:
family_status_df.loc[family_status_df.family_status_id == 1, 'family_status']

1    гражданский брак
Name: family_status, dtype: object

И наконец запросим оттуда же информацию по категории '4'`:

In [61]:
family_status_df.loc[family_status_df.family_status_id == 4, 'family_status']

4    Не женат / не замужем
Name: family_status, dtype: object

#### Выводы:

В данных представлено, что:
* большинство заёмщиков без кредитных задолженностей находятся в официальном браке;
* многие заёмщики состоят в гражданском браке или вовсе не состоят в отношениях;
* примерно одна десятая часть всех заёмщиков имеет задолженности, вне зависимости от семейного статуса.

Таким образом, гипотеза подтверждена частично:
* с одной стороны - люди, находящиеся в браке, действительно склонны возвращать кредиты в срок;
* с другой - одна десятая часть заёмщиков имеет кредитные задолженности, вне зависимости от вышеназванных категорий.

### Влияние уровня дохода на возврат кредита

Настала очередь третьей гипотезы:

*Средний или высокий уровень дохода также напрямую увеличивает вероятность погашения кредита*.

Построим ещё одну сводную таблицу. Передадим методу `pivot_table()` следующие аргументы:
* `total_income_category`- в качестве индексов
* `debt`- в качестве наименований столбцов
* `dob_years` - в качестве значений, которые будут отражены в таблице
* `count()` - в качестве агрегатной функции, которая будет применена к значениям таблицы

Таким образом мы рассчитаем количество людей с разным уровнем дохода, имеющих или не имеющих банковские задолженности:

In [62]:
income_level_pivot = df.pivot_table(index='total_income_category',
                            columns='debt',
                            values='dob_years',
                            aggfunc='count')
display(income_level_pivot)

debt,0,1
total_income_category,,
A,23,2
B,4686,356
C,14655,1360
D,329,21
E,20,2


Уровень дохода подавляющего большинства заёмщиков относится к категории 'C'. Если мы обратимся к **пункту 2.6**, то выясним, что в денежном эквиваленте он равносилен доходу от пятидесяти до двухсот тысяч включительно. Можно сказать, это средний уровень заработной платы по Центральному региону РФ.

#### Выводы:

Исходя из представленных данных:
* подавляющее большинство заёмщиков без кредитных задолженностей имеют средний уровень дохода;
* многие заёмщики имеют уровень дохода выше среднего;
* примерно одна десятая часть всех заёмщиков имеет задолженности, вне зависимости от уровня доходов.

Таким образом, гипотеза подтверждена частично:
* с одной стороны - средний уровень дохода влияет на факт погашения кредита в срок;
* с другой - высокий уровень дохода не оказывает на это сильного влияния;
* и помимо этого, одна десятая часть заёмщиков имеет кредитные задолженности, вне зависимости от вышеназванных категорий.

### Влияние целей кредитования на возврат кредита

И наконец настала очередь последней гипотезы:

*Кредиты на образовательные цели и на покупку недвижимости с большей вероятностью будут выплачены в срок, нежели кредиты на автомобили и на проведение свадьбы*.

Выведем на экран очередную сводную таблицу. Передадим методу `pivot_table()` следующие аргументы:
* `purpose_category`- в качестве индексов
* `debt`- в качестве наименований столбцов
* `dob_years` - в качестве значений, которые будут отражены в таблице
* `count()` - в качестве агрегатной функции, которая будет применена к значениям таблицы

Таким образом мы рассчитаем количество людей с разными целями кредитования, имеющих или не имеющих банковские задолженности:

In [63]:
lending_purpose_pivot = df.pivot_table(index='purpose_category',
                            columns='debt',
                            values='dob_years',
                            aggfunc='count')
display(lending_purpose_pivot)

debt,0,1
purpose_category,,
операции с автомобилем,3903,403
операции с недвижимостью,10029,782
получение образования,3643,370
проведение свадьбы,2138,186


#### Выводы:

В данных прослеживаются закономерности:
* большинство кредитов оформляются на недвижимость (ипотека) и выплачиваются в срок;
* примерно одна десятая часть всех заёмщиков имеет задолженности, вне зависимости от категории кредитования.

Т.е. наша гипотеза подтверждается частично:
* ипотечное кредитование имеет больше шансов на своевременный возврат кредита (вероятно, это связано с длительным сроком его предоставления);
* одна десятая часть заёмщиков имеет кредитные задолженности, вне зависимости от вышеназванных категорий.

## Итоги исследования:

Мы проверили четыре гипотезы и установили:

1. Количество детей у заёмщика не оказывает влияния на факт своевременного закрытия кредита.

    Первая гипотеза была опровергнута.
    
    
2. Люди, находящиеся в зарегистрированном браке, действительно склонны возвращать кредиты в срок.
    
    Вторая гипотеза была частично подтверждена.
    
    
3. Средний уровень дохода также напрямую увеличивает вероятность погашения кредита.
    
    Третья гипотеза была частично подтверждена.
    
    
4. Ипотечные кредиты с большей вероятностью будут выплачены в срок.

    Четвёртая гипотеза была частично подтверждена.
    
На основании предоставленных данных мы можем нарисовать условный "портрет" типичного благонадёжного заёмщика: это человек, имеющий средний уровень дохода, состоящий в официальном браке, без детей, который оформляет ипотечное кредитование. Соответствие любой из вышеназванных характеристик может рассматриваться как условное подтверждение благонадёжности заёмщика.

С другой стороны, в модели кредитного скоринга непременно должен быть учтён факт того, что одна десятая часть всех заёмщиков вне зависимости от каких-либо категорий имеет задолженности по кредитам. Проведённое выше исследование это полностью подтверждает. Это - довольно значительное количество людей, поэтому банку рекомендуется спланировать политику по снижению данной тенденции.